In [1]:
import pandas as pd
import os
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
import numpy as np
def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()
%matplotlib inline

test = pd.read_csv('../input/test.csv')

In [2]:
# # Loop through each file. Make sure it's only the type that is saved in the results file
# for fold in tqdm(os.listdir('../type_results/')):
#     for file in os.listdir(f'../type_results/{fold}'):
#         if 'sub' in file:
#             df = df.loc[df['type'] == fold]
#             df.to_parquet(f'../type_results/{fold}/{file}')

In [50]:
def save_res_types_folder(type_, oof, sub, fi, file):
    """
    Saves the oof, sub, and fi files int he type_results folder with correct naming convention
    """
    model_name_runid = file.split('_oof')[0]
    for exist_file in os.listdir(f'../type_results/{type_}/'):
        # Check to see if results already list
        if model_name_runid in exist_file:
            if 'LMAE' is exist_file:
                print('file {exist_file} already exists')
                return

    oof_type = oof.loc[oof['type'] == type_]
    score = mean_absolute_error(oof_type['scalar_coupling_constant'],oof_type['oof_preds'])
    logscore = np.log(score)
    if score > 1:
        print(f'No predictions for {type_}')
        return
    print(f'===== running for type {type_} - mae {score} - log mae {logscore}')
    
    oof_type = oof.loc[oof['type'] == type_]
    
    sub_type = test[['id','molecule_name','type']].merge(sub, on='id')
    sub_type = sub_type.loc[sub_type['type'] == type_]
    if np.sum(sub_type['scalar_coupling_constant'] == 0) > 10:
        print('ERROR! Sub has to many zero predictions')
        return
    expected_len = len(test.loc[test['type'] == type_])
    if expected_len != len(sub_type):
        print("ERRROR LENGTHS NOT THE SAME")
        return
        
    # Name Files and save
    model_type = file.split('_oof')[1].split('_')[1]
    nfolds = [x for x in file.split('_oof')[1].split('_') if 'fold' in x][0]
    niter = [x for x in file.split('_oof')[1].split('_') if 'iter' in x][0]
    lr = [x for x in file.split('_oof')[1].split('_') if 'lr' in x][0].replace('.csv','')
    fn_template = '../type_results/{}/{}_{}_XXXXXXX_{:0.4f}MAE_{:0.4}LMAE_{}_{}_{}_{}.parquet'.format(type_,
                                                                                                      model_name_runid,
                                                                                                      type_,
                                                                                                      score,
                                                                                                      logscore,
                                                                                                      model_type,
                                                                                                      nfolds,
                                                                                                       niter,
                                                                                                       lr)
    sub_name = fn_template.replace('XXXXXXX','submission')
    oof_name = fn_template.replace('XXXXXXX','oof')

    print(sub_name)
    print(oof_name)
    sub_type.to_parquet(sub_name)
    oof_type.to_parquet(oof_name)

    if fi is not None:
        fi_type = fi.loc[fi['type'] == type_]
        fi_name = fn_template.replace('XXXXXXX','fi')
        print(fi_name)
        fi_type.to_parquet(fi_name)

In [5]:
# types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC','3JHH','3JHC', '3JHN']

# for file in tqdm(os.listdir('../oof/')):
#     if 'M04' in file or 'M03' in file:
#         print(f'====== Running for file {file}')
#         if 'MERGE' not in file:
#             print(file)
#             oof = pd.read_csv(f'../oof/{file}')
#             sub = pd.read_csv('../submissions/{}'.format(file.replace('oof','submission').replace('catboost','lgb')))
#             for fi_file in os.listdir('../fi/'):
#                 if file.split('_oof')[0] in fi_file:
#                     #print(f'found it {fi_file}')
#                     try:
#                         fi = pd.read_csv(f'../fi/{fi_file}')
#                     except:
#                         print(f'Cant read fi file {fi_file}')
#             for type_ in types:
#                 save_res_types_folder(type_, oof, sub, fi, file)

  0%|          | 0/66 [00:00<?, ?it/s]

====== Running for file M046_2JHC_0713_0745_oof_catboost_2folds_1.6653CV_500000iter_0.1lr.csv
M046_2JHC_0713_0745_oof_catboost_2folds_1.6653CV_500000iter_0.1lr.csv
No predictions for 1JHC
No predictions for 2JHH
No predictions for 1JHN
No predictions for 2JHN
===== running for type 2JHC - mae 0.1957371996255439 - log mae -1.6309823376897268
../type_results/2JHC/M046_2JHC_0713_0745_2JHC_submission_0.1957MAE_-1.631LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHC/M046_2JHC_0713_0745_2JHC_oof_0.1957MAE_-1.631LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHC/M046_2JHC_0713_0745_2JHC_fi_0.1957MAE_-1.631LMAE_catboost_2folds_500000iter_0.1lr.parquet
No predictions for 3JHH
No predictions for 3JHC


  8%|▊         | 5/66 [00:06<01:16,  1.26s/it]

No predictions for 3JHN
====== Running for file M039_0708_1415_oof_catboost_2folds_-1.4994CV_50000iter_0.1lr.csv
M039_0708_1415_oof_catboost_2folds_-1.4994CV_50000iter_0.1lr.csv
===== running for type 1JHC - mae 0.6020701708134761 - log mae -0.5073812776531496
../type_results/1JHC/M039_0708_1415_1JHC_submission_0.6021MAE_-0.5074LMAE_catboost_2folds_50000iter_0.1lr.parquet
../type_results/1JHC/M039_0708_1415_1JHC_oof_0.6021MAE_-0.5074LMAE_catboost_2folds_50000iter_0.1lr.parquet
../type_results/1JHC/M039_0708_1415_1JHC_fi_0.6021MAE_-0.5074LMAE_catboost_2folds_50000iter_0.1lr.parquet
===== running for type 2JHH - mae 0.1440483484191191 - log mae -1.9376062828479823
../type_results/2JHH/M039_0708_1415_2JHH_submission_0.1440MAE_-1.938LMAE_catboost_2folds_50000iter_0.1lr.parquet
../type_results/2JHH/M039_0708_1415_2JHH_oof_0.1440MAE_-1.938LMAE_catboost_2folds_50000iter_0.1lr.parquet
../type_results/2JHH/M039_0708_1415_2JHH_fi_0.1440MAE_-1.938LMAE_catboost_2folds_50000iter_0.1lr.parquet
=====

  9%|▉         | 6/66 [00:23<06:09,  6.15s/it]

...

No predictions for 3JHC
No predictions for 3JHN
====== Running for file M033_0704_1133_oof_lgb_2folds_1.4555CV_5000000iter_0.1lr.csv
M033_0704_1133_oof_lgb_2folds_1.4555CV_5000000iter_0.1lr.csv
No predictions for 1JHC
No predictions for 2JHH
No predictions for 1JHN
No predictions for 2JHN
No predictions for 2JHC
No predictions for 3JHH
No predictions for 3JHC
===== running for type 3JHN - mae 0.6455157127220198 - log mae -0.43770572705208766


 97%|█████████▋| 64/66 [03:13<00:08,  4.01s/it]

../type_results/3JHN/M033_0704_1133_3JHN_submission_0.6455MAE_-0.4377LMAE_lgb_2folds_5000000iter_0.1lr.parquet
../type_results/3JHN/M033_0704_1133_3JHN_oof_0.6455MAE_-0.4377LMAE_lgb_2folds_5000000iter_0.1lr.parquet
../type_results/3JHN/M033_0704_1133_3JHN_fi_0.6455MAE_-0.4377LMAE_lgb_2folds_5000000iter_0.1lr.parquet
====== Running for file M040_0708_1639_oof_catboost_2folds_-1.5419CV_500000iter_0.1lr.csv
M040_0708_1639_oof_catboost_2folds_-1.5419CV_500000iter_0.1lr.csv
===== running for type 1JHC - mae 0.5782496288908177 - log mae -0.5477496196506169
../type_results/1JHC/M040_0708_1639_1JHC_submission_0.5782MAE_-0.5477LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/1JHC/M040_0708_1639_1JHC_oof_0.5782MAE_-0.5477LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/1JHC/M040_0708_1639_1JHC_fi_0.5782MAE_-0.5477LMAE_catboost_2folds_500000iter_0.1lr.parquet
===== running for type 2JHH - mae 0.14214817296391355 - log mae -1.9508852938420602
../type_results/2JHH/M040_070

100%|██████████| 66/66 [03:29<00:00,  5.32s/it]

../type_results/3JHN/M040_0708_1639_3JHN_submission_0.1158MAE_-2.156LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/3JHN/M040_0708_1639_3JHN_oof_0.1158MAE_-2.156LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/3JHN/M040_0708_1639_3JHN_fi_0.1158MAE_-2.156LMAE_catboost_2folds_500000iter_0.1lr.parquet


In [35]:
# for folder in tqdm(os.listdir('../temp/')):
#     for file in os.listdir(f'../temp/{folder}/'):
#         if 'M04' in file or 'M03' in file:
#             if 'oof' in file:
#                 for type_ in types:
#                     oof = pd.read_csv(f'../temp/{folder}/{file}') 
#                     sub = pd.read_csv('../temp/{}/{}'.format(folder, file.replace('oof','submission')))
#                     try:
#                         fi = pd.read_csv('../temp/{}/{}'.format(folder, file.replace('oof','fi'))) 
#                     except:
#                         print('No feature importance')
#                         fi = None
#                     file_new = '_'.join(file.split('_')[1:])
#                     save_res_types_folder(type_, oof, sub, fi, file_new)


  0%|          | 0/43 [00:00<?, ?it/s]

No predictions for 1JHC
No predictions for 2JHH
No predictions for 1JHN
No predictions for 2JHN
No predictions for 2JHC
No predictions for 3JHH
No predictions for 3JHC
===== running for type 3JHN - mae 0.12409500418646358 - log mae -2.086707843935925
../type_results/3JHN/M041_0708_1847_3JHN_submission_0.1241MAE_-2.087LMAE_lgbm_2folds_300000iter_0.1lr.parquet
../type_results/3JHN/M041_0708_1847_3JHN_oof_0.1241MAE_-2.087LMAE_lgbm_2folds_300000iter_0.1lr.parquet
../type_results/3JHN/M041_0708_1847_3JHN_fi_0.1241MAE_-2.087LMAE_lgbm_2folds_300000iter_0.1lr.parquet
No predictions for 1JHC
No predictions for 2JHH
No predictions for 1JHN
No predictions for 2JHN
===== running for type 2JHC - mae 0.23482589370418944 - log mae -1.44891091727812
../type_results/2JHC/M041_0708_1847_2JHC_submission_0.2348MAE_-1.449LMAE_lgbm_2folds_300000iter_0.1lr.parquet
../type_results/2JHC/M041_0708_1847_2JHC_oof_0.2348MAE_-1.449LMAE_lgbm_2folds_300000iter_0.1lr.parquet
../type_results/2JHC/M041_0708_1847_2JHC_fi


  9%|▉         | 4/43 [00:44<07:18, 11.23s/it]

../type_results/3JHN/M041_0708_1847_3JHN_submission_0.1241MAE_-2.087LMAE_lgbm_2folds_300000iter_0.1lr.parquet
../type_results/3JHN/M041_0708_1847_3JHN_oof_0.1241MAE_-2.087LMAE_lgbm_2folds_300000iter_0.1lr.parquet
../type_results/3JHN/M041_0708_1847_3JHN_fi_0.1241MAE_-2.087LMAE_lgbm_2folds_300000iter_0.1lr.parquet
No predictions for 1JHC
===== running for type 2JHH - mae 0.12512516484694158 - log mae -2.0784407238895457
../type_results/2JHH/M046_0714_0836_2JHH_submission_0.1251MAE_-2.078LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHH/M046_0714_0836_2JHH_oof_0.1251MAE_-2.078LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHH/M046_0714_0836_2JHH_fi_0.1251MAE_-2.078LMAE_catboost_2folds_500000iter_0.1lr.parquet
No predictions for 1JHN
No predictions for 2JHN
No predictions for 2JHC
No predictions for 3JHH
No predictions for 3JHC



 23%|██▎       | 10/43 [01:05<04:54,  8.91s/it]

...


 40%|███▉      | 17/43 [10:30<33:37, 77.60s/it]

../type_results/3JHN/M040_0708_1639_3JHN_submission_0.1158MAE_-2.156LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/3JHN/M040_0708_1639_3JHN_oof_0.1158MAE_-2.156LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/3JHN/M040_0708_1639_3JHN_fi_0.1158MAE_-2.156LMAE_catboost_2folds_500000iter_0.1lr.parquet
No predictions for 1JHC
No predictions for 2JHH
No predictions for 1JHN
===== running for type 2JHN - mae 0.14187002914681504 - log mae -1.9528439282109176
../type_results/2JHN/M037_0706_2029_2JHN_submission_0.1419MAE_-1.953LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHN/M037_0706_2029_2JHN_oof_0.1419MAE_-1.953LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHN/M037_0706_2029_2JHN_fi_0.1419MAE_-1.953LMAE_catboost_2folds_500000iter_0.1lr.parquet
No predictions for 2JHC
No predictions for 3JHH
No predictions for 3JHC
No predictions for 3JHN
No predictions for 1JHC
No predictions for 2JHH
No predictions for 1JHN
===== running for type


 47%|████▋     | 20/43 [13:51<28:31, 74.40s/it]

No predictions for 3JHN


FileNotFoundError: [Errno 2] File b'../temp/M032/temp1of8_M032_0703_2133_fi_lgb_3folds_5000000iter_0.1lr.csv' does not exist: b'../temp/M032/temp1of8_M032_0703_2133_fi_lgb_3folds_5000000iter_0.1lr.csv'

In [51]:
# Pick up where we left off
for folder in tqdm(os.listdir('../temp/')[22:]):
    for file in os.listdir(f'../temp/{folder}/'):
        if 'M04' in file or 'M03' in file:
            if 'oof' in file:
                for type_ in types:
                    oof = pd.read_csv(f'../temp/{folder}/{file}') 
                    sub = pd.read_csv('../temp/{}/{}'.format(folder, file.replace('oof','submission')))
                    try:
                        fi = pd.read_csv('../temp/{}/{}'.format(folder, file.replace('oof','fi'))) 
                    except:
                        print('No feature importance')
                        fi = None
                    file_new = '_'.join(file.split('_')[1:])
                    save_res_types_folder(type_, oof, sub, fi, file_new)





  0%|          | 0/21 [00:00<?, ?it/s]

No feature importance
No predictions for 1JHC
No feature importance
No predictions for 2JHH
No feature importance
===== running for type 1JHN - mae 0.44270593599961294 - log mae -0.8148495307059426
../type_results/1JHN/M032_0703_2133_1JHN_submission_0.4427MAE_-0.8148LMAE_lgb_3folds_5000000iter_0.1lr.parquet
../type_results/1JHN/M032_0703_2133_1JHN_oof_0.4427MAE_-0.8148LMAE_lgb_3folds_5000000iter_0.1lr.parquet
No feature importance
No predictions for 2JHN
No feature importance
No predictions for 2JHC
No feature importance
No predictions for 3JHH
No feature importance
No predictions for 3JHC
No feature importance
No predictions for 3JHN
No feature importance
No predictions for 1JHC
No feature importance
===== running for type 2JHH - mae 0.16153666617745602 - log mae -1.82302312693348
../type_results/2JHH/M032_0703_2133_2JHH_submission_0.1615MAE_-1.823LMAE_lgb_3folds_5000000iter_0.1lr.parquet
../type_results/2JHH/M032_0703_2133_2JHH_oof_0.1615MAE_-1.823LMAE_lgb_3folds_5000000iter_0.1lr.pa





  5%|▍         | 1/21 [03:27<1:09:09, 207.46s/it]

No predictions for 3JHN
No predictions for 1JHC
===== running for type 2JHH - mae 0.13819274540620405 - log mae -1.9791058624692037
../type_results/2JHH/M043_0710_2124_2JHH_submission_0.1382MAE_-1.979LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHH/M043_0710_2124_2JHH_oof_0.1382MAE_-1.979LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHH/M043_0710_2124_2JHH_fi_0.1382MAE_-1.979LMAE_catboost_2folds_500000iter_0.1lr.parquet
===== running for type 1JHN - mae 0.37638296857357506 - log mae -0.9771481205051243
../type_results/1JHN/M043_0710_2124_1JHN_submission_0.3764MAE_-0.9771LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/1JHN/M043_0710_2124_1JHN_oof_0.3764MAE_-0.9771LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/1JHN/M043_0710_2124_1JHN_fi_0.3764MAE_-0.9771LMAE_catboost_2folds_500000iter_0.1lr.parquet
===== running for type 2JHN - mae 0.14780459847175392 - log mae -1.9118641581526112
../type_results/2JHN/M043_0710_2124_2JHN_su





 14%|█▍        | 3/21 [06:47<52:32, 175.16s/it]  

...

No predictions for 3JHN
No predictions for 1JHC
===== running for type 2JHH - mae 0.17040615127316805 - log mae -1.7695705662175365
../type_results/2JHH/M035_0704_2344_2JHH_submission_0.1704MAE_-1.77LMAE_lgbm_2folds_500000iter_0.01lr.parquet
../type_results/2JHH/M035_0704_2344_2JHH_oof_0.1704MAE_-1.77LMAE_lgbm_2folds_500000iter_0.01lr.parquet
../type_results/2JHH/M035_0704_2344_2JHH_fi_0.1704MAE_-1.77LMAE_lgbm_2folds_500000iter_0.01lr.parquet
===== running for type 1JHN - mae 0.3845589756904514 - log mae -0.9556581189883623
../type_results/1JHN/M035_0704_2344_1JHN_submission_0.3846MAE_-0.9557LMAE_lgbm_2folds_500000iter_0.01lr.parquet
../type_results/1JHN/M035_0704_2344_1JHN_oof_0.3846MAE_-0.9557LMAE_lgbm_2folds_500000iter_0.01lr.parquet
../type_results/1JHN/M035_0704_2344_1JHN_fi_0.3846MAE_-0.9557LMAE_lgbm_2folds_500000iter_0.01lr.parquet
===== running for type 2JHN - mae 0.15149590307866517 - log mae -1.8871966967834135
../type_results/2JHN/M035_0704_2344_2JHN_submission_0.1515MAE_-1.





 90%|█████████ | 19/21 [24:52<03:31, 105.87s/it]

No predictions for 3JHN
No predictions for 1JHC
===== running for type 2JHH - mae 0.1440483484191191 - log mae -1.9376062828479823
../type_results/2JHH/M039_0708_1415_2JHH_submission_0.1440MAE_-1.938LMAE_catboost_2folds_50000iter_0.1lr.parquet
../type_results/2JHH/M039_0708_1415_2JHH_oof_0.1440MAE_-1.938LMAE_catboost_2folds_50000iter_0.1lr.parquet
../type_results/2JHH/M039_0708_1415_2JHH_fi_0.1440MAE_-1.938LMAE_catboost_2folds_50000iter_0.1lr.parquet
===== running for type 1JHN - mae 0.44416797387173707 - log mae -0.8115524685637627
../type_results/1JHN/M039_0708_1415_1JHN_submission_0.4442MAE_-0.8116LMAE_catboost_2folds_50000iter_0.1lr.parquet
../type_results/1JHN/M039_0708_1415_1JHN_oof_0.4442MAE_-0.8116LMAE_catboost_2folds_50000iter_0.1lr.parquet
../type_results/1JHN/M039_0708_1415_1JHN_fi_0.4442MAE_-0.8116LMAE_catboost_2folds_50000iter_0.1lr.parquet
===== running for type 2JHN - mae 0.1435319947223771 - log mae -1.9411973088964702
../type_results/2JHN/M039_0708_1415_2JHN_submission





 95%|█████████▌| 20/21 [28:56<02:27, 147.31s/it]



100%|██████████| 21/21 [28:56<00:00, 82.68s/it] 

../type_results/3JHN/M039_0708_1415_3JHN_submission_0.1160MAE_-2.155LMAE_catboost_2folds_50000iter_0.1lr.parquet
../type_results/3JHN/M039_0708_1415_3JHN_oof_0.1160MAE_-2.155LMAE_catboost_2folds_50000iter_0.1lr.parquet
../type_results/3JHN/M039_0708_1415_3JHN_fi_0.1160MAE_-2.155LMAE_catboost_2folds_50000iter_0.1lr.parquet


In [52]:
types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC','3JHH','3JHC', '3JHN']

for file in tqdm(os.listdir('../oof/')):
    if 'M046' in file:
        print(f'====== Running for file {file}')
        if 'MERGE' not in file:
            print(file)
            oof = pd.read_csv(f'../oof/{file}')
            sub = pd.read_csv('../submissions/{}'.format(file.replace('oof','submission').replace('catboost','lgb')))
            for fi_file in os.listdir('../fi/'):
                if file.split('_oof')[0] in fi_file:
                    #print(f'found it {fi_file}')
                    try:
                        fi = pd.read_csv(f'../fi/{fi_file}')
                    except:
                        print(f'Cant read fi file {fi_file}')
            for type_ in types:
                save_res_types_folder(type_, oof, sub, fi, file)





  0%|          | 0/67 [00:00<?, ?it/s]

====== Running for file M046_2JHC_0713_0745_oof_catboost_2folds_1.6653CV_500000iter_0.1lr.csv
M046_2JHC_0713_0745_oof_catboost_2folds_1.6653CV_500000iter_0.1lr.csv
No predictions for 1JHC
No predictions for 2JHH
No predictions for 1JHN
No predictions for 2JHN
===== running for type 2JHC - mae 0.1957371996255439 - log mae -1.6309823376897268
../type_results/2JHC/M046_2JHC_0713_0745_2JHC_submission_0.1957MAE_-1.631LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHC/M046_2JHC_0713_0745_2JHC_oof_0.1957MAE_-1.631LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHC/M046_2JHC_0713_0745_2JHC_fi_0.1957MAE_-1.631LMAE_catboost_2folds_500000iter_0.1lr.parquet
No predictions for 3JHH
No predictions for 3JHC






  7%|▋         | 5/67 [00:06<01:16,  1.24s/it]

No predictions for 3JHN
====== Running for file M046_1JHC_0712_1944_oof_catboost_2folds_1.3679CV_10000iter_0.1lr.csv
M046_1JHC_0712_1944_oof_catboost_2folds_1.3679CV_10000iter_0.1lr.csv
===== running for type 1JHC - mae 0.6172682111875856 - log mae -0.48245164744548047
../type_results/1JHC/M046_1JHC_0712_1944_1JHC_submission_0.6173MAE_-0.4825LMAE_catboost_2folds_10000iter_0.1lr.parquet
../type_results/1JHC/M046_1JHC_0712_1944_1JHC_oof_0.6173MAE_-0.4825LMAE_catboost_2folds_10000iter_0.1lr.parquet
../type_results/1JHC/M046_1JHC_0712_1944_1JHC_fi_0.6173MAE_-0.4825LMAE_catboost_2folds_10000iter_0.1lr.parquet
No predictions for 2JHH
No predictions for 1JHN
No predictions for 2JHN
No predictions for 2JHC
No predictions for 3JHH
No predictions for 3JHC






 18%|█▊        | 12/67 [00:11<01:00,  1.10s/it]

...

No predictions for 3JHN
====== Running for file M046_3JHC_0713_1412_oof_catboost_2folds_1.6446CV_500000iter_0.1lr.csv
M046_3JHC_0713_1412_oof_catboost_2folds_1.6446CV_500000iter_0.1lr.csv
No predictions for 1JHC
No predictions for 2JHH
No predictions for 1JHN
No predictions for 2JHN
No predictions for 2JHC
No predictions for 3JHH
===== running for type 3JHC - mae 0.2216202227144778 - log mae -1.5067900705359292
../type_results/3JHC/M046_3JHC_0713_1412_3JHC_submission_0.2216MAE_-1.507LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/3JHC/M046_3JHC_0713_1412_3JHC_oof_0.2216MAE_-1.507LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/3JHC/M046_3JHC_0713_1412_3JHC_fi_0.2216MAE_-1.507LMAE_catboost_2folds_500000iter_0.1lr.parquet






 79%|███████▉  | 53/67 [00:42<00:19,  1.40s/it]

No predictions for 3JHN
====== Running for file M046_2JHN_0714_0349_oof_catboost_2folds_1.5922CV_500000iter_0.1lr.csv
M046_2JHN_0714_0349_oof_catboost_2folds_1.5922CV_500000iter_0.1lr.csv
No predictions for 1JHC
No predictions for 2JHH
No predictions for 1JHN
===== running for type 2JHN - mae 0.1300098094987309 - log mae -2.040145373690797
../type_results/2JHN/M046_2JHN_0714_0349_2JHN_submission_0.1300MAE_-2.04LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHN/M046_2JHN_0714_0349_2JHN_oof_0.1300MAE_-2.04LMAE_catboost_2folds_500000iter_0.1lr.parquet
../type_results/2JHN/M046_2JHN_0714_0349_2JHN_fi_0.1300MAE_-2.04LMAE_catboost_2folds_500000iter_0.1lr.parquet
No predictions for 2JHC
No predictions for 3JHH






 94%|█████████▍| 63/67 [00:47<00:04,  1.13s/it]



100%|██████████| 67/67 [00:47<00:00,  1.41it/s]

No predictions for 3JHC
No predictions for 3JHN
